In [1]:
import pandas as pd
import lightgbm as lgb

In [2]:
def get_all_teams():
    df_results = pd.read_csv("data/results.csv")
    df_results.tail()
    teams = df_results[(df_results["tournament"] == "UEFA Euro") & (df_results["date"].str.contains("2024"))][["home_team","away_team"]].dropna()
    return set(teams["home_team"]) | set(teams["away_team"])

teams = get_all_teams()
teams

{'Albania',
 'Austria',
 'Belgium',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'England',
 'France',
 'Georgia',
 'Germany',
 'Hungary',
 'Italy',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Scotland',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Switzerland',
 'Turkey',
 'Ukraine'}

In [3]:
df_iso = pd.read_csv("data/iso.csv")
print([i for i in df_iso["English short name lower case"] if ("e" in i.lower())])

['Zimbabwe', 'Yemen', 'Western Sahara', 'Vietnam', 'Venezuela', 'Uzbekistan', 'United States Minor Outlying Islands', 'United States', 'United Kingdom', 'United Arab Emirates', 'Ukraine', 'Turkmenistan', 'Turkey', 'Tokelau', 'Timor-Leste', 'Syrian Arab Republic', 'Switzerland', 'Sweden', 'Svalbard and Jan Mayen', 'Suriname', 'South Korea', 'South Georgia and the South Sandwich Islands', 'Slovenia', 'Singapore', 'Sierra Leone', 'Seychelles', 'Serbia', 'Senegal', 'Sao Tome and Principe', 'Saint Vincent and the Grenadines', 'Saint Pierre and Miquelon', 'Saint Kitts and Nevis', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Barthélemy', 'Reunion', 'Puerto Rico', 'Philippines', 'Peru', 'Papua New Guinea', 'Palestinian Territory, Occupied', 'Northern Mariana Islands', 'Niue', 'Nigeria', 'Niger', 'New Zealand', 'New Caledonia', 'Netherlands Antilles', 'Netherlands', 'Nepal', 'Mozambique', 'Montserrat', 'Montenegro', 'Moldova, Republic of', 'Micronesia, Federated States of', 'Mexico', 

In [7]:
def get_elos():
    df_iso = pd.read_csv("data/iso.csv")
    df_elo = pd.read_csv("data/elo.csv", index_col=0)
    
    df_iso = df_iso[df_iso["English short name lower case"].isin(teams)].rename(columns={
        "English short name lower case":"country",
        "Alpha-2 code": "code"
    })[["country","code"]]
    
    return {
        row["country"]: (row["code"], float(df_elo.loc[row["code"], "2024"]))
        for _, row in df_iso.iterrows()
    }
    
get_elos()

{'Ukraine': ('UA', 1867.0),
 'Turkey': ('TR', 1757.0),
 'Switzerland': ('CH', 1806.0),
 'Spain': ('ES', 2019.0),
 'Slovenia': ('SI', 1741.0),
 'Slovakia': ('SK', 1655.0),
 'Serbia': ('RS', 1783.0),
 'Romania': ('RO', 1657.0),
 'Portugal': ('PT', 2015.0),
 'Poland': ('PL', 1721.0),
 'Netherlands': ('NL', 1968.0),
 'Italy': ('IT', 1949.0),
 'Hungary': ('HU', 1828.0),
 'Germany': ('DE', 1917.0),
 'Georgia': ('GE', 1650.0),
 'France': ('FR', 2086.0),
 'Denmark': ('DK', 1826.0),
 'Czech Republic': ('CZ', 1772.0),
 'Croatia': ('HR', 1955.0),
 'Belgium': ('BE', 1987.0),
 'Austria': ('AT', 1862.0),
 'Albania': ('AL', 1614.0),
 'England': ('EN', 2000.0),
 'Scotland': ('SQ', 1776.0)}

In [8]:
loaded_model = lgb.Booster(model_file='lightgbm_model.txt')

In [12]:
loaded_model.predict(

Signature:
loaded_model.predict(
    data: Union[str, pathlib.Path, numpy.ndarray, pandas.core.frame.DataFrame, lightgbm.compat.dt_DataTable, scipy.sparse._matrix.spmatrix, pyarrow.lib.Table],
    start_iteration: int = 0,
    num_iteration: Optional[int] = None,
    raw_score: bool = False,
    pred_leaf: bool = False,
    pred_contrib: bool = False,
    data_has_header: bool = False,
    validate_features: bool = False,
    **kwargs: Any,
) -> Union[numpy.ndarray, scipy.sparse._matrix.spmatrix, List[scipy.sparse._matrix.spmatrix]]
Docstring:
Make a prediction.

Parameters
----------
data : str, pathlib.Path, numpy array, pandas DataFrame, pyarrow Table, H2O DataTable's Frame or scipy.sparse
    Data source for prediction.
    If str or pathlib.Path, it represents the path to a text file (CSV, TSV, or LibSVM).
start_iteration : int, optional (default=0)
    Start index of the iteration to predict.
    If <= 0, starts from the first iteration.
num_iteration : int or None, optional (def